In [5]:
import numpy as np
import pandas as pd

In [6]:
print("hello!")

hello!


# General instructions
Instruct students to go to [Google Colab](https://colab.research.google.com/). 
A popup should appear to "Open notebook". Under GitHub, enter the main repo url.

In [7]:
df_data = pd.read_csv("./data/data.csv")

In [8]:
df_data.head()

,species,prop1,prop2
0,Ag,5.24,3.28
1,Al,6.75,7.32
